# Analyze probing results
- Preprocess a bit. Select the best results from each configuration. Export to the same directory.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
def analyze_results(report_dir):
    dfs = []
    for p in Path(report_dir).iterdir():
        if not p.name.startswith("report"): 
            continue
        dfs.append(pd.read_csv(p))
    df = pd.concat(dfs, axis=0)
    tasks = []
    layers = []
    for i, row in df.iterrows():
        tasks.append("_".join(row["task"].split("_")[:-2]))
        layers.append(int(row["task"].split("_")[-1]))
    df.drop(columns=["task"], inplace=True)
    df["task"] = tasks
    df["layer"] = layers
    best_results = df.sort_values(by=["val_acc"], ascending=False).groupby(by=["task", "config", "layer"]).head(1)
    #best_results.to_csv(Path(report_dir, "best_results.csv"), index=False)
    return best_results 


all_model_best_results = []
for p in Path("../reports").iterdir():
    df_best_results = analyze_results(p)
    df_best_results["LM"] = [p.name] * len(df_best_results)
    all_model_best_results.append(df_best_results) 
df_all = pd.concat(all_model_best_results, axis=0)

In [4]:
print(df_all.shape)
df_all.head()

(1638, 14)


,train_acc,train_loss,val_acc,val_loss,test_acc,test_loss,model,rs,config,train_size_per_class,nclasses,task,layer,LM
1311,0.889167,0.276470,0.888333,0.320556,0.860000,0.387374,MLP-20,2,Nonzero,1200,2,past_present,12,embeddings_roberta_base_corr_1000
1302,0.880417,0.311666,0.886667,0.331587,0.856667,0.390802,LogReg,2,Full,1200,2,past_present,12,embeddings_roberta_base_corr_1000
674,0.853750,0.368538,0.875000,0.353258,0.853333,0.408543,MLP-20,2,Full,1200,2,past_present,6,embeddings_roberta_base_corr_1000
589,0.869167,0.359074,0.873333,0.376625,0.876667,0.348882,MLP-10,31,Full,1200,2,past_present,5,embeddings_roberta_base_corr_1000
576,0.862083,0.370171,0.868333,0.366434,0.831667,0.421208,MLP-20,2,Nonzero,1200,2,past_present,5,embeddings_roberta_base_corr_1000


In [8]:
df_all.to_csv("../reports/probing_roberta_base.csv", index=False)